In [1]:
!cat data1/matrix.txt | head -n 5

0,0,893480
0,1,602014
0,2,161301
0,3,310639
0,4,758183
cat: write error: Broken pipe


In [6]:
!hdfs dfs -rm -R /outputs
!hdfs dfs -rm -R /inputs

Deleted /outputs
Deleted /inputs


In [7]:
!hadoop fs -mkdir -p /outputs
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data1 /inputs

2020-10-21 21:35:13,787 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2020-10-21 21:35:19,143 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


In [8]:
!hadoop dfs -ls /inputs/data1


Found 2 items
-rw-r--r--   3 root supergroup   46785000 2020-10-21 21:35 /inputs/data1/matrix.txt
-rw-r--r--   3 root supergroup         85 2020-10-21 21:35 /inputs/data1/smatrix.txt


In [9]:
%%writefile mapper.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*
import sys
import string

for line in sys.stdin: # reads from stdin
    line = line.strip()
    row, col, val = line.split(",")
    print(row+"\t"+col+","+val)

Overwriting mapper.py


In [10]:
%%writefile reducer.py
#!/opt/bitnami/python/bin/python
# -*-coding:utf-8 -*

import sys
import random

for line in sys.stdin:
    line = line.strip()
    row, colval = line.split()
    col, val = colval.split(",")
    print(col+" "+row+" "+val)

Overwriting reducer.py


In [11]:
!cat data1/smatrix.txt | python mapper.py | python reducer.py

0 0 1
1 0 2
2 0 3
3 0 4
0 1 5
1 1 6
2 1 7
3 2 8
0 2 9
1 2 10
2 2 11
3 2 12


# Small Matrix

In [12]:
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D mapreduce.job.reduces=3 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data1/smatrix.txt \
    -output /outputs/result1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

2020-10-21 21:36:25,072 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/mapper.py, /training/reducer.py, /tmp/hadoop-unjar2874859834494374989/] [] /tmp/streamjob7512282812789142905.jar tmpDir=null
2020-10-21 21:36:28,805 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.4:8032
2020-10-21 21:36:30,118 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2020-10-21 21:36:30,410 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.4:8032
2020-10-21 21:36:30,410 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2020-10-21 21:36:31,880 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1603314927371_0001
2020-10-21 21:36:33,091 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = fals

In [11]:
!hdfs dfs -rm -R /outputs/result1

Deleted /outputs/result1


In [13]:
!hdfs dfs -ls /outputs/result1

Found 4 items
-rw-r--r--   3 root supergroup          0 2020-10-21 21:41 /outputs/result1/_SUCCESS
-rw-r--r--   3 root supergroup         28 2020-10-21 21:41 /outputs/result1/part-00000
-rw-r--r--   3 root supergroup         21 2020-10-21 21:41 /outputs/result1/part-00001
-rw-r--r--   3 root supergroup         38 2020-10-21 21:41 /outputs/result1/part-00002


In [14]:
!hdfs dfs -cat /outputs/result1/part-00000

2020-10-21 21:50:48,093 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
3 0 4	
2 0 3	
1 0 2	
0 0 1	


In [15]:
!hdfs dfs -cat /outputs/result1/part-00001

2020-10-21 21:50:53,926 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2 1 7	
1 1 6	
0 1 5	


In [16]:
!hdfs dfs -cat /outputs/result1/part-00002

2020-10-21 21:51:03,484 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
3 2 12	
2 2 11	
1 2 10	
0 2 9	
3 2 8	


In [17]:
!hdfs dfs -cat /outputs/result1/part-00003

cat: `/outputs/result1/part-00003': No such file or directory


# Big Matrix

In [18]:
!hdfs dfs -rm -R /outputs
!hadoop jar /opt/hadoop-3.2.1/share/hadoop/tools/lib/hadoop-streaming-3.2.1.jar \
    -D mapreduce.job.reduces=10 \
    -file $PWD/mapper.py\
    -file $PWD/reducer.py\
    -mapper mapper.py \
    -reducer reducer.py \
    -input /inputs/data1/matrix.txt \
    -output /outputs/result1 \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

Deleted /outputs
2020-10-21 21:51:34,930 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/training/mapper.py, /training/reducer.py, /tmp/hadoop-unjar1284866028686982399/] [] /tmp/streamjob9049864868948939990.jar tmpDir=null
2020-10-21 21:51:38,473 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.4:8032
2020-10-21 21:51:44,473 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2020-10-21 21:51:44,577 INFO client.RMProxy: Connecting to ResourceManager at resourcemanager/172.18.0.4:8032
2020-10-21 21:51:44,577 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.18.0.5:10200
2020-10-21 21:51:45,897 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1603314927371_0002
2020-10-21 21:51:46,881 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localH

In [19]:
!hdfs dfs -ls /outputs/result1

Found 11 items
-rw-r--r--   3 root supergroup          0 2020-10-21 21:58 /outputs/result1/_SUCCESS
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:56 /outputs/result1/part-00000
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:56 /outputs/result1/part-00001
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:56 /outputs/result1/part-00002
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:57 /outputs/result1/part-00003
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:57 /outputs/result1/part-00004
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:57 /outputs/result1/part-00005
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:57 /outputs/result1/part-00006
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:57 /outputs/result1/part-00007
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:58 /outputs/result1/part-00008
-rw-r--r--   3 root supergroup    4978500 2020-10-21 21:58 /outputs/result1/part-00009


In [20]:
!hdfs dfs -cat /outputs/result1/part-00001 | head

2020-10-21 22:00:45,228 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2902 106 173883	
5999 106 138702	
5998 106 843380	
5997 106 151487	
5996 106 328035	
5995 106 456071	
5994 106 305879	
5993 106 744884	
5992 106 479108	
5991 106 156839	
cat: Unable to write to output stream.


In [21]:
!hdfs dfs -cat /outputs/result1/part-00002 | tail

2020-10-21 22:00:52,726 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2912 97 654964	
2911 97 145313	
2910 97 199041	
2909 97 710664	
2908 97 370762	
2907 97 686962	
2906 97 606642	
2905 97 215659	
2904 97 376380	
2903 97 760455	
